# Warehouse Data Generation

In [45]:
# This code is used to generate and prepare synthetic data for the database in SimioWarehouseSystem

In [46]:
# import associated packages
from funcdefinations import *
import pyodbc
import pandas as pd

### Get Data Generation Parameters

In [47]:
# get the paremeters from excel file
# PickerNum --- number of pickers
# TransporterNum --- number of transporters
# NumOrders --- number of orders
# NumSKUs --- number of skus
# SKUWeights --- list about sku weighting
# LineOrderDist --- distribution for the number of lines in an order
# QuantityLineDist --- distribution for the quantity in an order line
PickerNum,TransporterNum, NumOrders, NumSKUs, SKUWeights, LineOrderDist, QuantityLineDist = readFromExcel(FileName = 'ParameterSetting.xlsx', SheetName = 'Parameter Setting', SKUWeightsSheet = 'SKU Weights')

# get dataframe of the graph representation of the warehouse
Nodes = pd.read_csv('Nodes.csv')
Arcs = pd.read_csv('Arcs.csv')

# get the settings of the graph representation of the warehouse
# PickLNum --- number of picking locations
# HandOffNum --- number of handoff spots
PickLNum = len(Nodes[Nodes['Type']=='MyPickNode'])
HandOffNum = len(Nodes[Nodes['Type']=='MyCoopNode'])

# picker and pransporter settings
V_p = 1    # picker speed (m/s)
C_p = 3    # picker capacity(items)
Tc_p = 45    # picker place time(s/item)
V_t = 2    # transporter speed (m/s)
C_t = 6    # transporter capacity(items)
Tu_t = 60    # transporter unload time (s/stay)


### Generate data

In [48]:
# Generate the initial dataframes
InitializationParaList = [NumOrders, NumSKUs, PickLNum, SKUWeights, LineOrderDist, QuantityLineDist]
[OrderInitial, SKUInitial, LocationInitial, OrderSKUInitial, SKULocationInitial]=dataframesInitialization(InitializationParaList)

# Complete the dataframes
# Complete the order dataframe
FinalOrderParaList = [OrderInitial]
OrderTable = completeOrder(FinalOrderParaList)

# Complete the sku dataframe
FinalSKUParaList = [ SKULocationInitial]
SKUTable = completeSKU(FinalSKUParaList)

# Complete the order_sku dataframe
FinalOrderSKUParaList = [OrderSKUInitial, True]
LineItemTable = completeOrderSKU(FinalOrderSKUParaList)

## Generate Database Tables

In [49]:
# connect to the database
conn_str = 'DSN=JDATA;database=WarehouseModel;Trusted_Connection=yes;'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

### Nodes

In [50]:
# Nodes table contain the all nodes (PickingNodes, HandoffNodes, and transferNodes)
# clear existing table
cursor.execute("TRUNCATE TABLE Nodes;")
cursor.commit()
# write new tables
for index, row in Nodes.iterrows():
    cursor.execute("INSERT INTO Nodes (ID, Node, N_xloc, N_zloc, Type) VALUES ({:}, '{:}', {:}, {:},'{:}')".format(row['ID'], row['Node'], row['N_xloc'], row['N_zloc'], row['Type']) )
cursor.commit()

### Arcs

In [51]:
# clear existing table
cursor.execute("TRUNCATE TABLE Arcs;")
cursor.commit()
# write new tables
for index, row in Arcs.iterrows():
    cursor.execute("INSERT INTO Arcs (Path, HeadNode, TailNode, Type) VALUES ('{:}', '{:}', '{:}', '{:}')".format(row['Path'], row['HeadNode'], row['TailNode'], row['Type']) )
cursor.commit()

### Orders

In [52]:
# clear existing table
cursor.execute("TRUNCATE TABLE Orders;")
cursor.commit()
# write new tables
for index, row in OrderTable.iterrows():
    cursor.execute("INSERT INTO Orders (OrderID, ReleaseDate, DueDate, Wave, FinalDestination) VALUES ({:}, '{:}', '{:}', {:}, '{:}')".format(row['OrderID'], row['ReleaseDate'], row['DueDate'], row['Wave'], row['FinalDestination']) )
cursor.commit()

### SKUs

In [53]:
# clear existing table
cursor.execute("TRUNCATE TABLE Skus;")
cursor.commit()
# write new tables
for index, row in SKUTable.iterrows():
    cursor.execute("INSERT INTO Skus (SkuID, PickNodeID, Weight) VALUES ({:}, {:}, {:})".format(row['SkuID'], row['PickNodeID'], row['Weight']) )
cursor.commit()

### LineItem

In [54]:
# clear existing table
cursor.execute("TRUNCATE TABLE LineItem;")
cursor.commit()
# write new tables
for index, row in LineItemTable.iterrows():
    cursor.execute("INSERT INTO LineItem (ItemID, OrderID, SkuID, Quantity) VALUES ({:}, {:}, {:}, {:})".format(index+1, row['OrderID'], row['SkuID'], row['Quantity']) )
cursor.commit()

### Pickers & Transporters

In [55]:
PcolName = 'Picker'
TcolName = 'Transporter'
# clear existing table
cursor.execute("TRUNCATE TABLE Pickers;")
cursor.commit()
# clear existing table
cursor.execute("TRUNCATE TABLE Transporters;")
cursor.commit()



# write new tables
for i in range(PickerNum):
    cursor.execute("INSERT INTO Pickers (PickerID, Picker, Speed, Capacity, PlaceTime) VALUES ({:}, '{:}', {:}, {:}, {:})".format(i+1, 'My'+ PcolName +str(i+1), V_p, C_p, Tc_p) )
cursor.commit()
for i in range(TransporterNum):
    cursor.execute("INSERT INTO Transporters (TransporterID, Transporter, Speed, Capacity, UnloadTime) VALUES ({:}, '{:}', {:}, {:}, {:})".format(i+1, 'My'+ TcolName +str(i+1), V_t, C_t, Tu_t) )
cursor.commit()